In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

# Bookmarks

## REGEXP / Extension Troubles

To find bookmarks, it is convenient to be able to use regular expresions when searching the database. This can be enabled in SQLite using an extension, which needs to be installed on the system first. See https://stackoverflow.com/a/8338515 and https://gist.github.com/janfri/a3e61731864a63554ba6f32bdc7179aa

Unfortunately, I haven't been able to load this extension using the %sql magic syntax, so whenever I need to use the REGEXP operator, I'll have to use the clumsy Python syntax directly.

In [8]:
con = sqlite3.connect("results.db")
con.enable_load_extension(True)
con.load_extension("pcre.so")

## Preliminary investigation

Aliases containing a forward slash:

In [48]:
%%sql 
with total as (select count(*) as total from alias)
select count(*) as '#', round(count(*)*100.0/total,2) as '%'
from alias 
join total 
where alias.value like '%/%'

 * sqlite:///results.db
Done.


#,%
451629,20.49


Arguments containing a forward slash:

In [49]:
%%sql 
with total as (select count(*) as total from argument)
select count(*) as '#', round(count(*)*100.0/total,2) as '%'
from argument 
join total 
where argument.name like '%/%'

 * sqlite:///results.db
Done.


#,%
418196,11.52


In [50]:
%%sql 
with total as (select count(*) as total from argument)
select name, count(*) as '#', round(count(*)*100.0/total,2) as '%'
from argument 
join total 
where argument.name like '%/%'
group by argument.name 
order by count(*) desc
limit 20;

 * sqlite:///results.db
Done.


name,#,%
"""$(history|tail -n1|sed -e '\''s/^\s*[0-9]\+\s*//;s/[;&|]\s*alert$//'\'')""",15722,0.43
~/.zshrc,13331,0.37
../..,10496,0.29
../../..,7764,0.21
~/.bashrc,7319,0.2
/dev/null,6073,0.17
../../../..,5089,0.14
~/.bash_profile,4403,0.12
~/.vimrc,3612,0.1
../,3115,0.09


Commands containing a forward slash:

In [51]:
%%sql 
with total as (select count(*) as total from command)
select count(*) as '#', round(count(*)*100.0/total,2) as '%'
from command 
join total 
where command.name like '%/%'

 * sqlite:///results.db
Done.


#,%
99423,3.92


In [52]:
%%sql 
with total as (select count(*) as total from command)
select name, count(*) as '#', round(count(*)*100.0/total,2) as '%'
from command 
join total 
where command.name like '%/%'
group by command.name 
order by count(*) desc
limit 20;

 * sqlite:///results.db
Done.


name,#,%
/usr/bin/git,3175,0.13
/usr/sbin/lsof,1707,0.07
/bin/ls,1631,0.06
/System/Library/CoreServices/Menu Extras/User.menu/Contents/Resources/CGSession,1218,0.05
/System/Library/Frameworks/CoreServices.framework/Frameworks/LaunchServices.framework/Support/lsregister,1174,0.05
/Applications/Google Chrome.app/Contents/MacOS/Google Chrome,854,0.03
/usr/local/bin/vim,848,0.03
../..,842,0.03
./gradlew,809,0.03
../../..,768,0.03


Aliases containing IPv4 adresses:

In [61]:
pd.read_sql_query('''
with total as (select count(*) as total from alias)
select count(*) as '#', round(count(*)*100.0/total,2) as '%'
from alias 
join total
where alias.value regexp '[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+'
''', con)

,#,%
0,17335,0.79


In [62]:
pd.read_sql_query('''
with total as (select count(*) as total from alias)
select alias.name, alias.value, count(*) as '#', round(count(*)*100.0/total,2) as '%'
from alias 
join total
where alias.value regexp '[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+'
group by alias.name, alias.value
order by count(*) desc
limit 20
''', con)

,name,value,#,%
0,portforward,"sudo ipfw add 1000 forward 127.0.0.1,3000 ip f...",93,0.0
1,p8,ping 8.8.8.8,69,0.0
2,localip,sudo ifconfig | grep -Eo 'inet (addr:)?([0-9]*...,66,0.0
3,djrun,dj runserver 0.0.0.0:8000,64,0.0
4,localip,sudo ifconfig | grep -Eo 'inet (addr:)?([0-9]*...,53,0.0
5,ping8,ping 8.8.8.8,40,0.0
6,p,ping 8.8.8.8,38,0.0
7,ts,thin start -a ${VM_IP:-127.0.0.1},36,0.0
8,vm,ssh vagrant@127.0.0.1 -p 2222,30,0.0
9,myip,ifconfig | sed -En 's/127.0.0.1//;s/.*inet (ad...,29,0.0


In [65]:
%sql select count(*) as '# 127.0.0.1' from alias where value like '%127.0.0.1%'

 * sqlite:///results.db
Done.


# 127.0.0.1
3045


Commands invoked with IPv4 adresses as arguments:

In [66]:
pd.read_sql_query('''
select command.name, count(*) as '#'
from command join argument using (command_id) 
where argument.name regexp '[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+' 
group by command.name 
order by count(*) 
desc limit 20
''', con)

,name,#
0,ssh,8498
1,ping,915
2,export,833
3,grep,523
4,docker,306
5,telnet,262
6,sshfs,249
7,curl,245
8,echo,221
9,nmap,199


Commands invoked with arugment containing forward slash:

In [67]:
%%sql 
with total as (select name, count(*) as total from command group by name)
select 
    command.name, 
    count(distinct command_id) as '#', 
    round(count(distinct command_id)*100.0/total,2) as '% with bookmark' 
from command join argument using (command_id)
join total using (name) 
where argument.name like '%/%' 
group by command.name 
order by count(*) desc 
limit 20

 * sqlite:///results.db
Done.


name,#,% with bookmark
cd,116017,69.62
vim,21845,46.93
source,19975,90.02
notify-send,15910,98.12
git,8162,2.49
rm,7814,29.29
open,7953,46.25
sed,6453,83.77
docker,3961,14.17
curl,6357,60.67


## Collecting Bookmarks

We will collect every argument that looks like a bookmark into a new table `bookmark`. The table consists only of the `argument_id` and a boolean field `remote` that indicates whether the bookmark points to a remote location.

Note: The dataset on Zenodo already includes this table.

In [70]:
%%sql
create table bookmark
( argument_id integer primary key
, remote integer not null
, foreign key (argument_id) references argument (argument_id)
);

 * sqlite:///results.db
Done.


[]

Remote bookmarks are either IP addresses or contain `.TLD:` or `.TLD/` or end with `.TLD`

In [73]:
con.execute('''
insert into bookmark (argument_id, remote) 
select argument_id, 1
from (
    select argument_id from argument where name regexp '([0-9]+\.[0-9]+\.[0-9]+\.[0-9]+)|(\.(aaa|aarp|abarth|abb|abbott|abbvie|abc|able|abogado|abudhabi|ac|academy|accenture|accountant|accountants|aco|actor|ad|adac|ads|adult|ae|aeg|aero|aetna|af|afamilycompany|afl|africa|ag|agakhan|agency|ai|aig|aigo|airbus|airforce|airtel|akdn|al|alfaromeo|alibaba|alipay|allfinanz|allstate|ally|alsace|alstom|am|americanexpress|americanfamily|amex|amfam|amica|amsterdam|analytics|android|anquan|anz|ao|aol|apartments|app|apple|aq|aquarelle|ar|arab|aramco|archi|army|arpa|art|arte|as|asda|asia|associates|at|athleta|attorney|au|auction|audi|audible|audio|auspost|author|auto|autos|avianca|aw|aws|ax|axa|az|azure|ba|baby|baidu|banamex|bananarepublic|band|bank|bar|barcelona|barclaycard|barclays|barefoot|bargains|baseball|basketball|bauhaus|bayern|bb|bbc|bbt|bbva|bcg|bcn|bd|be|beats|beauty|beer|bentley|berlin|best|bestbuy|bet|bf|bg|bh|bharti|bi|bible|bid|bike|bing|bingo|bio|biz|bj|black|blackfriday|blockbuster|blog|bloomberg|blue|bm|bms|bmw|bn|bnpparibas|bo|boats|boehringer|bofa|bom|bond|boo|book|booking|bosch|bostik|boston|bot|boutique|box|br|bradesco|bridgestone|broadway|broker|brother|brussels|bs|bt|budapest|bugatti|build|builders|business|buy|buzz|bv|bw|by|bz|bzh|ca|cab|cafe|cal|call|calvinklein|cam|camera|camp|cancerresearch|canon|capetown|capital|capitalone|car|caravan|cards|care|career|careers|cars|casa|case|caseih|cash|casino|cat|catering|catholic|cba|cbn|cbre|cbs|cc|cd|ceb|center|ceo|cern|cf|cfa|cfd|cg|ch|chanel|channel|charity|chase|chat|cheap|chintai|christmas|chrome|church|ci|cipriani|circle|cisco|citadel|citi|citic|city|cityeats|ck|cl|claims|cleaning|click|clinic|clinique|clothing|cloud|club|clubmed|cm|cn|co|coach|codes|coffee|college|cologne|com|comcast|commbank|community|company|compare|computer|comsec|condos|construction|consulting|contact|contractors|cooking|cookingchannel|cool|coop|corsica|country|coupon|coupons|courses|cpa|cr|credit|creditcard|creditunion|cricket|crown|crs|cruise|cruises|csc|cu|cuisinella|cv|cw|cx|cy|cymru|cyou|cz|dabur|dad|dance|data|date|dating|datsun|day|dclk|dds|de|deal|dealer|deals|degree|delivery|dell|deloitte|delta|democrat|dental|dentist|desi|design|dev|dhl|diamonds|diet|digital|direct|directory|discount|discover|dish|diy|dj|dk|dm|dnp|do|docs|doctor|dog|domains|dot|download|drive|dtv|dubai|duck|dunlop|dupont|durban|dvag|dvr|dz|earth|eat|ec|eco|edeka|edu|education|ee|eg|email|emerck|energy|engineer|engineering|enterprises|epson|equipment|er|ericsson|erni|es|esq|estate|esurance|et|etisalat|eu|eurovision|eus|events|exchange|expert|exposed|express|extraspace|fage|fail|fairwinds|faith|family|fan|fans|farm|farmers|fashion|fast|fedex|feedback|ferrari|ferrero|fi|fiat|fidelity|fido|film|final|finance|financial|fire|firestone|firmdale|fish|fishing|fit|fitness|fj|fk|flickr|flights|flir|florist|flowers|fly|fm|fo|foo|food|foodnetwork|football|ford|forex|forsale|forum|foundation|fox|fr|free|fresenius|frl|frogans|frontdoor|frontier|ftr|fujitsu|fujixerox|fun|fund|furniture|futbol|fyi|ga|gal|gallery|gallo|gallup|game|games|gap|garden|gay|gb|gbiz|gd|gdn|ge|gea|gent|genting|george|gf|gg|ggee|gh|gi|gift|gifts|gives|giving|gl|glade|glass|gle|global|globo|gm|gmail|gmbh|gmo|gmx|gn|godaddy|gold|goldpoint|golf|goo|goodyear|goog|google|gop|got|gov|gp|gq|gr|grainger|graphics|gratis|green|gripe|grocery|group|gs|gt|gu|guardian|gucci|guge|guide|guitars|guru|gw|gy|hair|hamburg|hangout|haus|hbo|hdfc|hdfcbank|health|healthcare|help|helsinki|here|hermes|hgtv|hiphop|hisamitsu|hitachi|hiv|hk|hkt|hm|hn|hockey|holdings|holiday|homedepot|homegoods|homes|homesense|honda|horse|hospital|host|hosting|hot|hoteles|hotels|hotmail|house|how|hr|hsbc|ht|hu|hughes|hyatt|hyundai|ibm|icbc|ice|icu|id|ie|ieee|ifm|ikano|il|im|imamat|imdb|immo|immobilien|in|inc|industries|infiniti|info|ing|ink|institute|insurance|insure|int|intel|international|intuit|investments|io|ipiranga|iq|ir|irish|is|ismaili|ist|istanbul|it|itau|itv|iveco|jaguar|java|jcb|jcp|je|jeep|jetzt|jewelry|jio|jll|jm|jmp|jnj|jo|jobs|joburg|jot|joy|jp|jpmorgan|jprs|juegos|juniper|kaufen|kddi|ke|kerryhotels|kerrylogistics|kerryproperties|kfh|kg|kh|ki|kia|kim|kinder|kindle|kitchen|kiwi|km|kn|koeln|komatsu|kosher|kp|kpmg|kpn|kr|krd|kred|kuokgroup|kw|ky|kyoto|kz|la|lacaixa|lamborghini|lamer|lancaster|lancia|land|landrover|lanxess|lasalle|lat|latino|latrobe|law|lawyer|lb|lc|lds|lease|leclerc|lefrak|legal|lego|lexus|lgbt|li|lidl|life|lifeinsurance|lifestyle|lighting|like|lilly|limited|limo|lincoln|linde|link|lipsy|live|living|lixil|lk|llc|llp|loan|loans|locker|locus|loft|lol|london|lotte|lotto|love|lpl|lplfinancial|lr|ls|lt|ltd|ltda|lu|lundbeck|lupin|luxe|luxury|lv|ly|ma|macys|madrid|maif|maison|makeup|man|management|mango|map|market|marketing|markets|marriott|marshalls|maserati|mattel|mba|mc|mckinsey|md|me|med|media|meet|melbourne|meme|memorial|men|menu|merckmsd|metlife|mg|mh|miami|microsoft|mil|mini|mint|mit|mitsubishi|mk|ml|mlb|mls|mm|mma|mn|mo|mobi|mobile|moda|moe|moi|mom|monash|money|monster|mormon|mortgage|moscow|moto|motorcycles|mov|movie|mp|mq|mr|ms|msd|mt|mtn|mtr|mu|museum|mutual|mv|mw|mx|my|mz|na|nab|nadex|nagoya|name|nationwide|natura|navy|nba|nc|ne|nec|net|netbank|netflix|network|neustar|new|newholland|news|next|nextdirect|nexus|nf|nfl|ng|ngo|nhk|ni|nico|nike|nikon|ninja|nissan|nissay|nl|no|nokia|northwesternmutual|norton|now|nowruz|nowtv|np|nr|nra|nrw|ntt|nu|nyc|nz|obi|observer|off|office|okinawa|olayan|olayangroup|oldnavy|ollo|om|omega|one|ong|onl|online|onyourside|ooo|open|oracle|orange|org|organic|origins|osaka|otsuka|ott|ovh|pa|page|panasonic|paris|pars|partners|parts|party|passagens|pay|pccw|pe|pet|pf|pfizer|pg|ph|pharmacy|phd|philips|phone|photo|photography|photos|physio|pics|pictet|pictures|pid|pin|ping|pink|pioneer|pizza|pk|pl|place|play|playstation|plumbing|plus|pm|pn|pnc|pohl|poker|politie|porn|post|pr|pramerica|praxi|press|prime|pro|prod|productions|prof|progressive|promo|properties|property|protection|pru|prudential|ps|pt|pub|pw|pwc|py|qa|qpon|quebec|quest|qvc|racing|radio|raid|re|read|realestate|realtor|realty|recipes|red|redstone|redumbrella|rehab|reise|reisen|reit|reliance|ren|rent|rentals|repair|report|republican|rest|restaurant|review|reviews|rexroth|rich|richardli|ricoh|rightathome|ril|rio|rip|rmit|ro|rocher|rocks|rodeo|rogers|room|rs|rsvp|ru|rugby|ruhr|run|rw|rwe|ryukyu|sa|saarland|safe|safety|sakura|sale|salon|samsclub|samsung|sandvik|sandvikcoromant|sanofi|sap|sarl|sas|save|saxo|sb|sbi|sbs|sc|sca|scb|schaeffler|schmidt|scholarships|school|schule|schwarz|science|scjohnson|scor|scot|sd|se|search|seat|secure|security|seek|select|sener|services|ses|seven|sew|sex|sexy|sfr|sg|sh|shangrila|sharp|shaw|shell|shia|shiksha|shoes|shop|shopping|shouji|show|showtime|shriram|si|silk|sina|singles|site|sj|sk|ski|skin|sky|skype|sl|sling|sm|smart|smile|sn|sncf|so|soccer|social|softbank|software|sohu|solar|solutions|song|sony|soy|space|sport|spot|spreadbetting|sr|srl|ss|st|stada|staples|star|statebank|statefarm|stc|stcgroup|stockholm|storage|store|stream|studio|study|style|su|sucks|supplies|supply|support|surf|surgery|suzuki|sv|swatch|swiftcover|swiss|sx|sy|sydney|symantec|systems|sz|tab|taipei|talk|taobao|target|tatamotors|tatar|tattoo|tax|taxi|tc|tci|td|tdk|team|tech|technology|tel|temasek|tennis|teva|tf|tg|th|thd|theater|theatre|tiaa|tickets|tienda|tiffany|tips|tires|tirol|tj|tjmaxx|tjx|tk|tkmaxx|tl|tm|tmall|tn|to|today|tokyo|tools|top|toray|toshiba|total|tours|town|toyota|toys|tr|trade|trading|training|travel|travelchannel|travelers|travelersinsurance|trust|trv|tt|tube|tui|tunes|tushu|tv|tvs|tw|tz|ua|ubank|ubs|ug|uk|unicom|university|uno|uol|ups|us|uy|uz|va|vacations|vana|vanguard|vc|ve|vegas|ventures|verisign|versicherung|vet|vg|vi|viajes|video|vig|viking|villas|vin|vip|virgin|visa|vision|vistaprint|viva|vivo|vlaanderen|vn|vodka|volkswagen|volvo|vote|voting|voto|voyage|vu|vuelos|wales|walmart|walter|wang|wanggou|watch|watches|weather|weatherchannel|webcam|weber|website|wed|wedding|weibo|weir|wf|whoswho|wien|wiki|williamhill|win|windows|wine|winners|wme|wolterskluwer|woodside|work|works|world|wow|ws|wtc|wtf|xbox|xerox|xfinity|xihuan|xin|xn--11b4c3d|xn--1ck2e1b|xn--1qqw23a|xn--2scrj9c|xn--30rr7y|xn--3bst00m|xn--3ds443g|xn--3e0b707e|xn--3hcrj9c|xn--3oq18vl8pn36a|xn--3pxu8k|xn--42c2d9a|xn--45br5cyl|xn--45brj9c|xn--45q11c|xn--4gbrim|xn--54b7fta0cc|xn--55qw42g|xn--55qx5d|xn--5su34j936bgsg|xn--5tzm5g|xn--6frz82g|xn--6qq986b3xl|xn--80adxhks|xn--80ao21a|xn--80aqecdr1a|xn--80asehdb|xn--80aswg|xn--8y0a063a|xn--90a3ac|xn--90ae|xn--90ais|xn--9dbq2a|xn--9et52u|xn--9krt00a|xn--b4w605ferd|xn--bck1b9a5dre4c|xn--c1avg|xn--c2br7g|xn--cck2b3b|xn--cg4bki|xn--clchc0ea0b2g2a9gcd|xn--czr694b|xn--czrs0t|xn--czru2d|xn--d1acj3b|xn--d1alf|xn--e1a4c|xn--eckvdtc9d|xn--efvy88h|xn--estv75g|xn--fct429k|xn--fhbei|xn--fiq228c5hs|xn--fiq64b|xn--fiqs8s|xn--fiqz9s|xn--fjq720a|xn--flw351e|xn--fpcrj9c3d|xn--fzc2c9e2c|xn--fzys8d69uvgm|xn--g2xx48c|xn--gckr3f0f|xn--gecrj9c|xn--gk3at1e|xn--h2breg3eve|xn--h2brj9c|xn--h2brj9c8c|xn--hxt814e|xn--i1b6b1a6a2e|xn--imr513n|xn--io0a7i|xn--j1aef|xn--j1amh|xn--j6w193g|xn--jlq61u9w7b|xn--jvr189m|xn--kcrx77d1x4a|xn--kprw13d|xn--kpry57d|xn--kpu716f|xn--kput3i|xn--l1acc|xn--lgbbat1ad8j|xn--mgb9awbf|xn--mgba3a3ejt|xn--mgba3a4f16a|xn--mgba7c0bbn0a|xn--mgbaakc7dvf|xn--mgbaam7a8h|xn--mgbab2bd|xn--mgbah1a3hjkrd|xn--mgbai9azgqp6j|xn--mgbayh7gpa|xn--mgbbh1a|xn--mgbbh1a71e|xn--mgbc0a9azcg|xn--mgbca7dzdo|xn--mgberp4a5d4ar|xn--mgbgu82a|xn--mgbi4ecexp|xn--mgbpl2fh|xn--mgbt3dhd|xn--mgbtx2b|xn--mgbx4cd0ab|xn--mix891f|xn--mk1bu44c|xn--mxtq1m|xn--ngbc5azd|xn--ngbe9e0a|xn--ngbrx|xn--node|xn--nqv7f|xn--nqv7fs00ema|xn--nyqy26a|xn--o3cw4h|xn--ogbpf8fl|xn--otu796d|xn--p1acf|xn--p1ai|xn--pbt977c|xn--pgbs0dh|xn--pssy2u|xn--q9jyb4c|xn--qcka1pmc|xn--qxa6a|xn--qxam|xn--rhqv96g|xn--rovu88b|xn--rvc1e0am3e|xn--s9brj9c|xn--ses554g|xn--t60b56a|xn--tckwe|xn--tiq49xqyj|xn--unup4y|xn--vermgensberater-ctb|xn--vermgensberatung-pwb|xn--vhquv|xn--vuq861b|xn--w4r85el8fhu5dnra|xn--w4rs40l|xn--wgbh1c|xn--wgbl6a|xn--xhq521b|xn--xkc2al3hye2a|xn--xkc2dl3a5ee0h|xn--y9a3aq|xn--yfro4i67o|xn--ygbi2ammx|xn--zfr164b|xxx|xyz|yachts|yahoo|yamaxun|yandex|ye|yodobashi|yoga|yokohama|you|youtube|yt|yun|za|zappos|zara|zero|zip|zm|zone|zuerich|zw))(:|/|$)'
)
''')

In [75]:
con.commit()

Local bookmarks are paths; paths contain `/`

In [76]:
%%sql
insert or ignore into bookmark (argument_id, remote)
select argument_id, 0
from (
    select argument_id from argument where name like '%/%'
)

 * sqlite:///results.db
374104 rows affected.


[]

In [77]:
%sql select count(*) from bookmark

 * sqlite:///results.db
Done.


count(*)
468267


### Look for false positives and define exclusion criteria

In [ ]:
%%sql
select argument.name, count(*)
from bookmark join argument using (argument_id)
group by argument.name
order by count(*) desc
limit 200;

In [ ]:
%%sql
delete from bookmark
where argument_id in 
(select argument_id from bookmark join argument using (argument_id) join command using (command_id)
 where argument.name in ('*/', '*(-/DN)', 'user.name', 'user.email')
or argument.name like '%/dev/null%'
or (command.name = 'git' and (
    argument.name like '%origin/%' 
    or argument.name like '%upstream/%' 
    or argument.name like '%refs/%'))
or argument.name like '%./*.deb%'
or (argument.name like "%'s/%" or argument.name like '%"s/%' or argument.name like '% s/%')
or argument.name like '%sub(/inet6%'
or argument.name like '%PATH//:/%'
or argument.name like '%(%%v/%%V)%'
or argument.name like '%/(\d+\.\d+\.\d+\.\d+)/%'
or argument.name like '%s;[^/]*/;%'
)

In [ ]:
con.execute('''
delete from bookmark
where argument_id in 
(select argument_id from bookmark join argument using (argument_id)
where argument.name regexp '^[\.\/]+$'
)
''')
con.commit()

Fix some TLDs to not count for remote bookmarks

In [ ]:
%%sql
update bookmark
set remote = 0
where argument_id in
(select argument_id from bookmark join argument using (argument_id)
 where argument.name like '%.app%'
 or argument.name like '%.pub%'
 or argument.name like '%.libs%'
 or argument.name like '%.menu%'
 or argument.name like '%.py%'
 or argument.name like '%.sh%'
)

## Analysis

In [26]:
%%sql 
with total as (select count(*) as total from alias)
select count(distinct alias_id) as '#', round(count(distinct alias_id)*100.0/total,2) as '%' 
from bookmark join argument using (argument_id) join command using (command_id) join alias using (alias_id)
join total

 * sqlite:///results.db
Done.


#,%
321546,14.59


In [27]:
%%sql
with 
    total as (select count(*) as total from alias),
    total_bookmarks as (select count(*) as total_bookmarks from bookmark)
select 
    count(distinct alias_id) as '# remote', 
    round(count(distinct alias_id)*100.0/total,2) as '%',
    round(count(distinct alias_id)*100.0/total_bookmarks,2) as '% of bookmarks'
from bookmark join argument using (argument_id) join command using (command_id) join alias using (alias_id)
join total
join total_bookmarks
where bookmark.remote = 1

 * sqlite:///results.db
Done.


# remote,%,% of bookmarks
59931,2.72,15.92


In [28]:
%%sql
select argument.name, count(*) as '#'
from argument join bookmark using (argument_id)
where bookmark.remote = 0
group by argument.name
order by count(*) desc
limit 20;

 * sqlite:///results.db
Done.


name,#
~/.zshrc,13331
~/.bashrc,7319
~/.bash_profile,4403
~/.vimrc,3612
manage.py,2374
~/Downloads,2169
~/.oh-my-zsh,1986
~/Desktop,1874
/etc/hosts,1769
/System/Library/CoreServices/Finder.app,1551


In [30]:
%%sql
select argument.name, count(*) as '#'
from argument join bookmark using (argument_id)
where bookmark.remote = 1
group by argument.name
order by count(*) desc
limit 20;

 * sqlite:///results.db
Done.


name,#
myip.opendns.com,2670
@resolver1.opendns.com,2645
8.8.8.8,706
127.0.0.1,643
google.com,461
wttr.in,422
towel.blinkenlights.nl,388
icanhazip.com,375
0.0.0.0,358
http://ipecho.net/plain,353
